In [2]:
import numpy as np
import json
import os
import sys 
import csv

In [3]:
label_set = np.loadtxt('./data/custom_labels.csv', delimiter=',', dtype='str')

In [4]:
datafiles_dir = './data/datafiles'

In [5]:
if os.path.exists(datafiles_dir) == False:
    os.mkdir(datafiles_dir)

In [6]:
meta = np.loadtxt('./data/custom_data/custom_meta_ncd.csv', delimiter=',', dtype=str, skiprows = 1)

In [7]:
meta

array([['693048.wav', '4', '0', 'lesred1', '20.0', '1 1 1 1'],
       ['692820.wav', '4', '0', 'lesred1', '23.312875', '1 1 1 1 1'],
       ['692819.wav', '5', '0', 'lesred1', '17.504875', '1 1 1 1'],
       ...,
       ['559833.wav', '5', '19', 'sposan', '19.45225', '1 1 1 1'],
       ['254627.wav', '4', '19', 'sposan', '1.700625', '0'],
       ['211173.wav', '5', '19', 'sposan', '3.8355', '1']], dtype='<U2565')

In [34]:
meta[3][2]

'0'

In [12]:
x = ['a', 'ab', 'c']
x[1:]

['ab', 'c']

In [15]:
nocall_label = 'a020'
for fold in [1,2,3,4,5]:
    base_path = "/datasets/xeno_canto/sm_dataset/"
    #meta = np.loadtxt('./data/custom_data/custom_meta_duration_probs.csv', delimiter=',', dtype='str', skiprows=1)
    
    train_wav_list = []
    eval_wav_list = []
    
    for i in range(0, len(meta)):   # range(0, len(meta)) 200 was small dataset
        cur_labels = [x for x in meta[i][2].split("-")]
        cur_labels = ['a'+x.zfill(3) for x in cur_labels]
        if len(cur_labels) > 0:
            sec_labels = cur_labels[1:]
            print(sec_labels)
        else:
            sec_labels = 0
        # indices shifted by 1
        #cur_path = (meta[i][1]).split('.')[0] + '.pt'
        cur_path = meta[i][0]
        
        cur_fold = int(meta[i][1])
        curr_call_detection = (meta[i][5]).split()
        for cd_ind, cd_value in enumerate (curr_call_detection):

            if cd_value == '1':
                cur_dict = {"wav": base_path + cur_path, "labels": "-".join(cur_labels), "sec_labels":sec_labels,"segment":cd_ind,}
                
                if cur_fold == fold:
                    eval_wav_list.append(cur_dict)
                else:
                    train_wav_list.append(cur_dict)
            elif cd_value == '0':
                cur_dict = {"wav": base_path + cur_path, "labels": nocall_label ,"segment":cd_ind} #
                if cur_fold == fold:
                    eval_wav_list.append(cur_dict)
                else:
                    train_wav_list.append(cur_dict)
        
    print('fold {:d}: {:d} training samples, {:d} test samples'.format(fold, len(train_wav_list), len(eval_wav_list)))

    with open(datafiles_dir + '/custom_train_data_'+ str(fold) +'.json', 'w') as f:
        json.dump({'data': train_wav_list}, f, indent=1)

    with open(datafiles_dir + '/custom_eval_data_'+ str(fold) +'.json', 'w') as f:
        json.dump({'data': eval_wav_list}, f, indent=1)   
print('Finished custom Preparation')

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['a002']
['a002']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [5]:
def make_name2mid_dict(label_csv):
    name_lookup = {}
    with open(label_csv, 'r') as f:
        csv_reader = csv.DictReader(f)
        line_count = 0
        for row in csv_reader:
            name_lookup[row['display_name']] = row['mid']
            line_count += 1
    return name_lookup

In [6]:
mid_dict = make_name2mid_dict('./data/custom_labels.csv')

In [7]:
mid_dict

{'lesred1': 'a000',
 'comred': 'a001',
 'hoared': 'a002',
 'shikra1': 'a003',
 'levspa1': 'a004',
 'norgos': 'a005',
 'eurspa1': 'a006',
 'cremyn': 'a007',
 'commyn': 'a008',
 'padwar1': 'a009',
 'grrwar1': 'a010',
 'afrwar1': 'a011',
 'blrwar1': 'a012',
 'mouwar1': 'a013',
 'aquwar1': 'a014',
 'marwar3': 'a015',
 'sedwar1': 'a016',
 'eurwar1': 'a017',
 'comsan': 'a018',
 'sposan': 'a019',
 'nocall': 'a020'}

In [10]:
for i in range(0, len(meta)):
    for sec in eval(meta[i][0]): 
                cur_sec_labels.append(mid_dict[sec])

SyntaxError: unexpected EOF while parsing (<string>, line 1)